### 프로젝트 변경점
- 사전 학습된 ResNet50을 사용(기존 v1 > v2로 변경)
- optimizer 변경(기존 SGD > AdamW로 변경)
- 학습률 스케줄러 추가

### 프로젝트 결과 분석
- 가정: 모델이 충분히 학습이 되었다는 전제하에 CAM과 Grad GAM의 결과를 분석하기
- CAM과 Grad CAM의 결과는 동일하게 나타나는 것을 볼 수 있다.
    -  이유: 현재 ResNet 구조를 보면, 마지막 Layer에서 AVG(1,1,c)를 사용하고 있기 때문에 GAP을 사용한 것과 같은 결과가 나온다.  
    만약 ResNet의 구조를 변경한다면, CAM과 Grad CAM에서 다른 결과를 확인할 수 있을 것이다. 하지만 CAM과 Grad GAM의 결과를 Ablation Study 관점에서 비교하기 어렵다는 문제가 있다.

- 여기서 주의깊게 살펴볼 것은 Grad CAM이다.  
    - 입력층과 가까운 저차원 레이어는 에지나 텍스처 같은 국소적인 기하학적 특징을 추출하며, 이는 히트맵과 바운딩 박스를 통해 모델이 이미지의 전반적인 기초 윤곽을 탐색하고 있음을 보여준다.  
    - 반대로 출력층에 가까운 레이어(고차원)는 레이어를 거치며 극대화된 Receptive Field을 통해 이미지 전체의 문맥을 통합적으로 수용하며, 고차원의 의미론적 추론을 바탕으로 압축된 특징 맵 속에서 객체의 본질적 형상에만 정보를 집중시켜 인식의 범위를 정교하게 수렴시킨다.

- 사전 학습된 ResNet 모델의 학습 과정에서 훈련 손실($L_{train}$)과 정확도는 최적점으로 점근적 수렴을 보이나, 검증(Validation) 지표는 특정 시점 이후 최적화 정체기(Optimization Plateau)에 진입한 것을 알 수 있다.  

    특히 AdamW의 가중치 감쇠(Weight Decay) 기법이 검증 손실의 급격한 발산을 억제하여 전형적인 과대적합 양상은 나타나지 않지만, 학습이 지속될수록 모델이 객체의 본질적 형상이 아닌 훈련 데이터 특유의 비본질적 노이즈를 암기하게 된다. 이로 인해 수치상의 안정에도 불구하고 실질적인 객체 변별력이 약화되는 일반화 성능 저하가 발생하며, 결과적으로 에포크가 증가할수록 타겟 객체에 대한 탐지 정밀도가 하락하는 결과가 발생한 것을 알 수 있다.


### 회고 
입력층에 인접한 저차원 레이어에서 에지나 텍스처 같은 국소적 특징을 추출하고, 이를 계층적으로 결합하여 고차원 레이어에서 객체의 공간적 위상과 고차원적 의미론적 추론을 수행하는 과정이 인상적이었다.  
특히, 초기 레이어의 시각적 파편들이 여러 단계의 필터를 거치며 Receptive Field를 확장하고, 객체를 인식해나가는 과정에서 인공신경망이 단순한 연산을 넘어 인간의 추론 방식을 공학적으로 재현하고 있음을 알게 되었다.

In [40]:
# !pip install opencv-python, numpy, torch

In [41]:
# !wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
# !wget http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar
# !wget http://vision.stanford.edu/aditya86/ImageNetDogs/lists.tar
# !tar -xf images.tar -C ./
# !tar -xf annotation.tar -C ./
# !tar -xf lists.tar -C ./

# print("데이터 다운로드 완료!")

## 프로젝트: CAM을 만들고 평가해 보자

In [ ]:
import os
import re
import shutil
import scipy.io as sio
import numpy as np
import math
from tqdm.auto import tqdm
import random
import cv2
import PIL
import seaborn as sns
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torchvision.models import ResNet50_Weights
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

print(torch.__version__)
print(np.__version__)
print(cv2.__version__)
print(PIL.__version__)

/home/user/anaconda3/envs/torch_gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.10.0+cu128
2.4.2
4.13.0
12.1.0


In [2]:
max_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


#### 데이터 분류
- stanford_dogs 폴더가 없을 때 실행

In [3]:
# # 원본 데이터 경로 (압축 해제 후 폴더)
# images_dir = './Images'  # 이미지들이 위치한 폴더
# mat_dir = './' # 프로젝트 루트 디렉토리

# # .mat 파일 경로 (train_list.mat와 test_list.mat가 각각 같은 구조라고 가정)
# train_mat_path = os.path.join(mat_dir, 'train_list.mat')
# test_mat_path = os.path.join(mat_dir, 'test_list.mat')

# # .mat 파일 로드
# train_mat = sio.loadmat(train_mat_path)
# test_mat = sio.loadmat(test_mat_path)

# # train_mat와 test_mat 내부에 'file_list'와 'labels' 등이 있음
# train_file_list = train_mat['file_list']
# train_labels = train_mat['labels'].squeeze()  # (N,)
# test_file_list = test_mat['file_list']
# test_labels = test_mat['labels'].squeeze()

# # 최종적으로 ImageFolder 구조로 재구성할 대상 폴더 생성 (예: stanford_dogs/train, stanford_dogs/test)
# base_dir = 'stanford_dogs'
# train_dir = os.path.join(base_dir, 'train')
# test_dir = os.path.join(base_dir, 'test')

# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)

# def process_mat_file(file_list_array, split_dir):
#     """
#     .mat 파일에서 로드한 file_list_array를 사용해,
#     split_dir(예: train 혹은 test)에 클래스별 폴더를 생성하고 이미지를 복사합니다.
#     """
#     num_files = file_list_array.shape[0]
#     for idx in range(num_files):
#         # file_list_array[idx]는 보통 array([<파일경로>]) 형태입니다.
#         # 따라서, array([<파일경로>]).item()을 사용하면 실제 문자열을 얻을 수 있습니다.
#         file_path = file_list_array[idx][0].item()

#         # 혹시 bytes 타입이면 문자열로 디코딩
#         if isinstance(file_path, bytes):
#             file_path = file_path.decode('utf-8')

#         # 파일 경로 예시: 'n02116738-African_hunting_dog/n02116738_2988.jpg'
#         # 클래스 이름은 파일 경로의 최상위 폴더명 (예: 'n02116738-African_hunting_dog')
#         class_folder = file_path.split('/')[0]

#         # 대상 클래스 폴더 생성
#         dest_folder = os.path.join(split_dir, class_folder)
#         os.makedirs(dest_folder, exist_ok=True)

#         # 원본 이미지 경로: Images 폴더 아래에 file_path 위치
#         src_path = os.path.join(images_dir, file_path)
#         # 대상 이미지 경로: dest_folder 아래에 원본 파일명 그대로 복사
#         dest_path = os.path.join(dest_folder, os.path.basename(file_path))

#         # 파일 존재 여부 확인 후 복사
#         if os.path.exists(src_path):
#             shutil.copy(src_path, dest_path)
#         else:
#             print(f"File not found: {src_path}")

# print("Processing train set...")
# process_mat_file(train_file_list, train_dir)
# print("Processing test set...")
# process_mat_file(test_file_list, test_dir)

# print("Dataset reorganization complete.")

In [4]:
class StanfordDogsDatasetWithBBox(datasets.ImageFolder):
    def __init__(self, root, annotation_root, transform=None):
        super().__init__(root, transform=transform)
        self.annotation_root = annotation_root  # 예: '/content/Annotation'
        self.new_size = (224, 224)

    def __getitem__(self, index):
        image, label = super().__getitem__(index)
        path, _ = self.samples[index]

        rel_path = os.path.relpath(path, self.root)
        annot_filename = os.path.splitext(os.path.basename(rel_path))[0]
        annot_folder = os.path.dirname(rel_path)
        annot_path = os.path.join(self.annotation_root, annot_folder, annot_filename)

        # bbox 기본값 (예: [ymin, xmin, ymax, xmax])
        bbox = [0.0, 0.0, 0.0, 0.0]

        if os.path.exists(annot_path):
            try:
                # 확장자가 없지만 XML 형식의 파일이라고 가정하고 파싱
                tree = ET.parse(annot_path)
                root_xml = tree.getroot()
                # 첫 번째 object 태그에서 bndbox 정보를 읽음
                obj = root_xml.find('object')
                if obj is not None:
                    bndbox = obj.find('bndbox')
                    if bndbox is not None:
                        xmin = float(bndbox.find('xmin').text)
                        ymin = float(bndbox.find('ymin').text)
                        xmax = float(bndbox.find('xmax').text)
                        ymax = float(bndbox.find('ymax').text)
                        # XML 내 <size> 태그에서 원본 이미지 크기 획득
                        size = root_xml.find('size')
                        w = float(size.find('width').text)
                        h = float(size.find('height').text)
                        new_h, new_w = self.new_size
                        # bbox 좌표 순서: [ymin, xmin, ymax, xmax]
                        bbox = [xmin * (new_w / w), ymin * (new_h / h),
                                xmax * (new_w / w), ymax * (new_h / h)]
                    else:
                        print(f"bndbox 태그를 찾을 수 없습니다: {annot_path}")
                else:
                    print(f"object 태그를 찾을 수 없습니다: {annot_path}")
            except Exception as e:
                print(f"Error parsing {annot_path}: {e}")
        else:
            print(f"Annotation file not found: {annot_path}")

        return image, label, bbox

In [5]:
def unnormalize(img_tensor):
    """
    img_tensor: [C, H, W] 텐서 (전처리 상태, 예: normalization 적용됨)
    ImageNet 평균 및 표준편차를 이용하여 복원 (RGB 순서)
    """
    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])
    img = img_tensor.cpu().numpy().transpose(1, 2, 0)
    img = std * img + mean
    img = np.clip(img, 0, 1)
    img = np.uint8(255 * img)
    return img

In [6]:
def get_bbox(cam, threshold=0.01):
    coords = np.argwhere(cam > threshold)
    if coords.size == 0:
        return None
    # np.argwhere의 결과는 (row, col) 즉, (y, x) 순서입니다.
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    return (x_min, y_min, x_max, y_max)

In [7]:
def visualize_bbox_on_image(img, bbox, box_color=(255, 0, 0), thickness=2):
    # 원본 이미지 복사
    img_with_bbox = img.copy()
    if bbox is not None:
        x_min, y_min, x_max, y_max = bbox
        cv2.rectangle(img_with_bbox, (x_min, y_min), (x_max, y_max), box_color, thickness)
    else:
        print("활성화된 영역이 없습니다.")
    return img_with_bbox

In [8]:
def visualize_both_bbox_on_image(img, bbox, ground_truth, thickness=2):
    # 원본 이미지 복사
    img_with_bbox = img.copy()

    if bbox is not None:
        x_min, y_min, x_max, y_max = bbox
    # 그리기 로직 진행
    else:
        print("경고: 객체가 탐지되지 않아 바운딩 박스를 생성할 수 없습니다.")
        # 기본값 설정 또는 건너뛰기
        x_min, y_min, x_max, y_max = 0, 0, 0, 0
    cv2.rectangle(img_with_bbox, (x_min, y_min), (x_max, y_max), (255, 0, 0), thickness)

    x_min_t, y_min_t, x_max_t, y_max_t = ground_truth
    cv2.rectangle(img_with_bbox, (x_min_t, y_min_t), (x_max_t, y_max_t), (0, 255, 0), thickness)

    return img_with_bbox

In [9]:
def get_iou(gt_bbox, pred_bbox):
    iou = None
    # TODO: get iou between two bbox
    # bbox 중 하나라도 None이면 IoU 0.0
    if gt_bbox is None or  pred_bbox is None:
        return 0.0

    x_min1, y_min1, x_max1, y_max1 = gt_bbox
    x_min2, y_min2, x_max2, y_max2 =  pred_bbox

    # 두 bbox의 교집합 영역 좌표 계산
    x_min_inter = max(x_min1, x_min2)
    y_min_inter = max(y_min1, y_min2)
    x_max_inter = min(x_max1, x_max2)
    y_max_inter = min(y_max1, y_max2)

    # 교집합의 너비와 높이 (음수가 되지 않도록)
    inter_width = max(0, x_max_inter - x_min_inter)
    inter_height = max(0, y_max_inter - y_min_inter)
    inter_area = inter_width * inter_height

    # 각 bbox의 면적 계산
    area1 = (x_max1 - x_min1) * (y_max1 - y_min1)
    area2 = (x_max2 - x_min2) * (y_max2 - y_min2)

    # 합집합 면적: A ∪ B = A + B - A ∩ B
    union_area = area1 + area2 - inter_area
    if union_area <= 0:
        return 0.0

    iou = inter_area / union_area
    return iou

### CAM

In [10]:
def generate_cam(model, item):
    cam_image = None
    # TODO: generate cam image
    model.eval()
    features = []

    def hook(module, input, output):
        features.append(output.detach())
    hook_handle = model.layer4.register_forward_hook(hook)

    output = model(item)
    hook_handle.remove()

    pred_class = output.argmax(dim=1).item()

    fc_weights = model.fc.weight.data.to(item.device)

    fmap = features[0][0]
    cam_image = torch.zeros(fmap.shape[1:], dtype=torch.float32, device=item.device)

    for i, w in enumerate(fc_weights[pred_class]):
        cam_image += w * fmap[i, :, :]
    cam_image = cam_image.cpu().numpy()

    cam_image = np.maximum(cam_image, 0)
    cam_image = (cam_image - np.min(cam_image)) / (np.max(cam_image) - np.min(cam_image) + 1e-8)

    return cam_image

In [11]:
def get_cam(models, image, ground_bbox, item, start_epochs=1, step_epochs=2):
    num_epochs = len(models)
    rows = 3
    cols = num_epochs + 1 

    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
    
    orig_img = unnormalize(image)
    ground_truth = [int(x) for x in ground_bbox]

    axes[0, 0].imshow(orig_img)
    axes[0, 0].set_title("Original Image")
    axes[0, 0].axis("off")
    
    axes[1, 0].imshow(orig_img)
    axes[1, 0].set_title("Original (For Overlay Ref)")
    axes[1, 0].axis("off")
    
    gt_only_img = visualize_bbox_on_image(orig_img.copy(), ground_truth) if 'visualize_bbox_on_image' in globals() else orig_img
    axes[2, 0].imshow(gt_only_img)
    axes[2, 0].set_title("Ground Truth BBox")
    axes[2, 0].axis("off")

    for i, model in enumerate(models):
        col_idx = i + 1
        epoch_num = start_epochs + (i * step_epochs)
        
        cam = generate_cam(model, item)
        cam_resized = cv2.resize(cam, (orig_img.shape[1], orig_img.shape[0]))
        cam_bbox = get_bbox(cam_resized, threshold=0.5)
        
        axes[0, col_idx].imshow(cam)
        axes[0, col_idx].set_title(f"Epoch {epoch_num} CAM")
        axes[0, col_idx].axis("off")
        
        axes[1, col_idx].imshow(orig_img)
        axes[1, col_idx].imshow(cam_resized, cmap='jet', alpha=0.5) 
        axes[1, col_idx].set_title(f"Epoch {epoch_num} Overlay")
        axes[1, col_idx].axis("off")
        
        iou_score = get_iou(cam_bbox, ground_truth) 
        cam_img_bbox = visualize_both_bbox_on_image(orig_img.copy(), cam_bbox, ground_truth)
        
        axes[2, col_idx].imshow(cam_img_bbox)
        title_text = f"Epoch {epoch_num} IOU: {iou_score:.4f}" if iou_score is not None else f"Epoch {epoch_num} Compare"
        axes[2, col_idx].set_title(title_text)
        axes[2, col_idx].axis("off")

    plt.tight_layout()
    plt.show()

### grad cam

In [12]:
def generate_grad_cam(model,item, activation_layer= "layer4"):
    # TODO: generate grad_cam_image
    model.eval()
    features = {}
    gradients = {}

    # forward hook: 대상 레이어의 출력을 저장
    def forward_hook(module, input, output):
        features['value'] = output.detach()

    # backward hook: 대상 레이어의 gradient를 저장
    def backward_hook(module, grad_in, grad_out):
        gradients['value'] = grad_out[0].detach()

    # 모델 내에서 이름이  activation_layer과 일치하는 레이어 검색
    target_layer = dict(model.named_modules()).get( activation_layer, None)
    if target_layer is None:
        raise ValueError(f"Layer '{activation_layer}' not found in the model.")
    
    # hook 등록
    forward_handle = target_layer.register_forward_hook(forward_hook)
    backward_handle = target_layer.register_backward_hook(backward_hook)

    # 순전파 실행
    output = model(item)
    pred_class = output.argmax(dim=1).item()

    # 예측 클래스에 대해 backward 수행
    model.zero_grad()
    score = output[0, pred_class]
    score.backward()

    # hook 제거
    forward_handle.remove()
    backward_handle.remove()

    # 저장된 feature map과 gradient 추출 (shape: [C, H, W])
    fmap = features['value'][0]
    grads = gradients['value'][0]

    # 각 채널에 대해 gradient의 global average pooling 계산 (weight 역할)
    weights = torch.mean(grads, dim=(1, 2))

    # weighted sum: 각 채널의 feature map에 weight를 곱해 합산
    grad_cam_image = torch.zeros(fmap.shape[1:], dtype=torch.float32, device=fmap.device)
    for i, w in enumerate(weights):
        grad_cam_image += w * fmap[i, :, :]
    grad_cam_image = grad_cam_image.cpu().numpy()

    # ReLU 적용 및 정규화: 음수 값 제거 및 [0,1] 범위로 스케일링
    grad_cam_image = np.maximum(grad_cam_image, 0)
    grad_cam_image = (grad_cam_image - grad_cam_image.min()) / (grad_cam_image.max() - grad_cam_image.min() + 1e-8)

    return grad_cam_image

In [13]:
def get_grad_cam(models_list, image, ground_bbox, item, start_epochs, step_epochs):
    layers = ["layer1", "layer2", "layer3", "layer4"]
    num_epochs = len(models_list)
    num_layers = len(layers)
    
    rows = num_epochs * 3 
    cols = num_layers + 1 
    
    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
    
    orig_img = unnormalize(image)
    ground_truth = [int(x) for x in ground_bbox]
    
    gt_only_img = visualize_bbox_on_image(orig_img.copy(), ground_truth) if 'visualize_bbox_on_image' in globals() else orig_img

    for e_idx, model in enumerate(models_list):
        epoch_num = start_epochs + (e_idx * step_epochs)
        base_row = e_idx * 3
        
        axes[base_row, 0].imshow(orig_img)
        axes[base_row, 0].set_title(f"Ep {epoch_num}\nOrig Image")
        axes[base_row, 0].axis("off")
        
        axes[base_row + 1, 0].imshow(orig_img)
        axes[base_row + 1, 0].set_title(f"Ep {epoch_num}\nOverlay Ref")
        axes[base_row + 1, 0].axis("off")
        
        axes[base_row + 2, 0].imshow(gt_only_img)
        axes[base_row + 2, 0].set_title(f"Ep {epoch_num}\nGround Truth")
        axes[base_row + 2, 0].axis("off")

        for l_idx, layer_name in enumerate(layers):
            col_idx = l_idx + 1
            
            grad_cam = generate_grad_cam(model, item, activation_layer=layer_name)
            grad_cam_resized = cv2.resize(grad_cam, (orig_img.shape[1], orig_img.shape[0]))
            grad_cam_bbox = get_bbox(grad_cam_resized, threshold=0.5)
            
            axes[base_row, col_idx].imshow(grad_cam)
            axes[base_row, col_idx].set_title(f"Ep {epoch_num} {layer_name} CAM")
            axes[base_row, col_idx].axis("off")
            
            axes[base_row + 1, col_idx].imshow(orig_img)
            axes[base_row + 1, col_idx].imshow(grad_cam_resized, cmap='jet', alpha=0.5)
            axes[base_row + 1, col_idx].set_title(f"Ep {epoch_num} {layer_name} Overlay")
            axes[base_row + 1, col_idx].axis("off")
            
            iou_score = get_iou(grad_cam_bbox, ground_truth)
            grad_cam_img_bbox = visualize_both_bbox_on_image(orig_img.copy(), grad_cam_bbox, ground_truth)
            
            axes[base_row + 2, col_idx].imshow(grad_cam_img_bbox)
            title_text = f"{layer_name} IOU: {iou_score:.4f}" if iou_score is not None else f"{layer_name} Compare"
            axes[base_row + 2, col_idx].set_title(title_text)
            axes[base_row + 2, col_idx].axis("off")

    plt.tight_layout()
    plt.show()

In [14]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 데이터셋 경로 (재구성한 ImageFolder 형식)
train_dir = os.path.join('stanford_dogs', 'train')
test_dir = os.path.join('stanford_dogs', 'test')

# Annotation 폴더 경로 (예: '/root/Annotation')
annotation_dir = './Annotation'
# 커스텀 데이터셋 생성: image, label, bbox 반환
train_dataset = StanfordDogsDatasetWithBBox(root=train_dir, annotation_root=annotation_dir, transform=transform)
valid_dataset = StanfordDogsDatasetWithBBox(root=test_dir, annotation_root=annotation_dir, transform=transform)

batch_size = 12
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, drop_last= True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2, drop_last= True)

num_classes = len(train_dataset.classes)
print(f"Number of classes: {num_classes}")
print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(valid_dataset)}")

Number of classes: 120
Train samples: 12000
Test samples: 8580


In [15]:
# Annotation 폴더에서 데이터를 골라보세요
annotation_path = './Annotation/n02085620-Chihuahua/n02085620_10074'

# XML 파일 파싱
tree = ET.parse(annotation_path)
root = tree.getroot()

for elem in root.iter():
    print(f"{elem.tag}: {elem.text}")

annotation: 
	
folder: 02085620
filename: n02085620_10074
source: 
		
database: ImageNet database
size: 
		
width: 333
height: 500
depth: 3
segment: 0
object: 
		
name: Chihuahua
pose: Unspecified
truncated: 0
difficult: 0
bndbox: 
			
xmin: 25
ymin: 10
xmax: 276
ymax: 498


### 모델 학습

In [16]:
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=2, model_type= ''):
    # 결과 이미지를 저장할 폴더 생성
    save_dir = './results'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    total_steps = num_epochs * len(train_loader)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=1e-6)

    for epoch in range(num_epochs):

        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        print(f"\nEpoch {epoch+1}/{num_epochs} 시작")

        for batch_idx, (images, labels, _) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

            if (batch_idx + 1) % 100 == 0:
                print(f"  Step {batch_idx+1} - Loss: {loss.item():.4f}")

            scheduler.step()

        train_loss = running_loss / (batch_idx + 1)
        train_acc = correct / total
        print(f"Train - Loss: {train_loss:.4f}, Accuracy: {train_acc*100:.2f}%")

        # 검증 단계
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        all_labels = []
        all_preds = []
        all_probs = []

        with torch.no_grad():
            for images, labels, _ in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                running_loss += loss.item()
                
                _, preds = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (preds == labels).sum().item()
                
                # ROC AUC를 위해 Softmax로 확률값 계산
                probs = F.softmax(outputs, dim=1)
                
                # GPU 텐서를 CPU로 옮기고 NumPy 배열로 변환하여 리스트에 저장
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())

        # 전체 검증 데이터에 대한 지표 계산
        test_loss = running_loss / len(valid_loader)
        test_acc = correct / total
        
        # scikit-learn을 이용한 지표 산출 (다중 분류 기준으로 average='macro' 사용)
        y_true = np.array(all_labels)
        y_pred = np.array(all_preds)
        y_prob = np.array(all_probs)
        
        val_precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
        val_recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
        val_f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
        val_conf_matrix = confusion_matrix(y_true, y_pred)
        
        # ROC AUC (다중 분류 처리. 이진 분류일 경우 multi_class 옵션을 빼고 y_prob[:, 1]을 넣어야 할 수 있습니다)
        try:
            val_roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
        except ValueError:
            val_roc_auc = float('nan')
            
        print(f"Validation - Loss: {test_loss:.4f}, Accuracy: {test_acc*100:.2f}%")
        print(f"             Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1-score: {val_f1:.4f}")
        print(f"             ROC AUC: {val_roc_auc:.4f}")


        plt.figure(figsize=(24, 20)) 

        sns.heatmap(val_conf_matrix, annot=False, fmt='d', cmap='Blues',
                    xticklabels=5, yticklabels=5) 

        plt.title(f'Confusion Matrix (Large) - Epoch {epoch+1}', fontsize=20)
        plt.xlabel('Predicted Label', fontsize=16)
        plt.ylabel('True Label', fontsize=16)
        plt.xticks(rotation=90, fontsize=12)
        plt.yticks(rotation=0, fontsize=12)

        cm_save_path = os.path.join(save_dir, f'confusion_matrix{model_type}_e{epoch+1}.png')
        plt.savefig(cm_save_path, bbox_inches='tight', dpi=300)
        plt.close()

        conf_matrix_errors = val_conf_matrix.copy()
        np.fill_diagonal(conf_matrix_errors, 0)

        plt.figure(figsize=(20, 16)) 
        sns.heatmap(conf_matrix_errors, annot=False, cmap='Reds', xticklabels=5, yticklabels=5)
        
        plt.title(f'Confusion Matrix (Errors Only) - Epoch {epoch+1}', fontsize=20)
        plt.xlabel('Predicted Label', fontsize=16)
        plt.ylabel('True Label', fontsize=16)
        
        cm_save_path = os.path.join(save_dir, f'confusion_matrix_errors{model_type}_e{epoch+1}.png')
        plt.savefig(cm_save_path, bbox_inches='tight') # 잘리는 부분 없이 저장
        plt.close()
        # 모델 저장
        cam_model_path = f'./models/cam_model{model_type}_e{epoch+1}.pt'
        folder_path = os.path.dirname(cam_model_path)

        if folder_path and not os.path.exists(folder_path):
            os.makedirs(folder_path)

        torch.save(model, cam_model_path)

### ResNet50  - pre trained

In [17]:
# model = models.resnet50(pretrained=True)
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))
model = model.to(device)

In [18]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay = 0.05)

In [ ]:
%%time

train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=max_epochs, model_type = "_pre")


Epoch 1/10 시작
  Step 100 - Loss: 4.0608
  Step 200 - Loss: 2.8946
  Step 300 - Loss: 2.0338
  Step 400 - Loss: 2.0954
  Step 500 - Loss: 2.1589
  Step 600 - Loss: 1.7397
  Step 700 - Loss: 1.6869
  Step 800 - Loss: 1.5228
  Step 900 - Loss: 1.2155
  Step 1000 - Loss: 1.6163
Train - Loss: 2.2981, Accuracy: 58.65%
Validation - Loss: 1.4325, Accuracy: 79.21%
             Precision: 0.7945, Recall: 0.7867, F1-score: 0.7760
             ROC AUC: 0.9970

Epoch 2/10 시작
  Step 100 - Loss: 1.0193
  Step 200 - Loss: 1.3875
  Step 300 - Loss: 1.3552
  Step 400 - Loss: 1.0478
  Step 500 - Loss: 1.0604
  Step 600 - Loss: 1.0308
  Step 700 - Loss: 1.3586
  Step 800 - Loss: 1.1080
  Step 900 - Loss: 1.4823
  Step 1000 - Loss: 0.9837
Train - Loss: 1.2988, Accuracy: 83.92%
Validation - Loss: 1.3806, Accuracy: 80.89%
             Precision: 0.8091, Recall: 0.8018, F1-score: 0.7972
             ROC AUC: 0.9974

Epoch 3/10 시작
  Step 100 - Loss: 1.0955
  Step 200 - Loss: 1.1142
  Step 300 - Loss: 0.9961
 

### CAM, Grad CAM 체크

In [ ]:
# validation dataset 확인용
def show_dataset_grid(dataset, start_index=0, num_images=12, cols=4):
    """데이터셋의 이미지를 역정규화하여 격자 형태로 시각화합니다."""
    if start_index + num_images > len(dataset):
        num_images = len(dataset) - start_index

    rows = (num_images + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
    axes = np.atleast_1d(axes).flatten()

    for i in range(num_images):
        current_idx = start_index + i
        image_tensor, label, _ = dataset[current_idx]

        img_to_show = unnormalize(image_tensor)
        
        axes[i].imshow(img_to_show)
        axes[i].set_title(f"Idx: {current_idx} | Lbl: {label}")
        axes[i].axis("off")

    # 남은 빈 칸 제거
    for j in range(num_images, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

# show_dataset_grid(valid_dataset, start_index=0, num_images=500, cols=10)
# show_dataset_grid(valid_dataset, start_index=501, num_images=500, cols=10)

In [ ]:
start_epochs = 1
step_epochs = 3

models_list = []

for epoch in range(start_epochs, max_epochs + 1, step_epochs):
    model_path = f'./models/cam_model_pre_e{epoch}.pt'
    
    model = torch.load(model_path,  map_location=device, weights_only=False)
    model.eval() 
    
    models_list.append(model)

### 단일 객체일 경우

In [ ]:
image, label, ground_bbox = valid_dataset[0]
item = image.unsqueeze(0).to(device)
orig_img = unnormalize(image)

In [ ]:
get_cam(models_list, image, ground_bbox, item, start_epochs, step_epochs)

In [ ]:
get_grad_cam(models_list, image, ground_bbox, item, start_epochs, step_epochs)

### 여러 객체가 있을 때

In [ ]:
image, label, ground_bbox = valid_dataset[326]
item = image.unsqueeze(0).to(device)
orig_img = unnormalize(image)

In [ ]:
get_cam(models_list, image, ground_bbox, item, start_epochs, step_epochs)

In [ ]:
get_grad_cam(models_list, image, ground_bbox, item, start_epochs, step_epochs)

### 객체가 멀리 있을 때

In [ ]:
image, label, ground_bbox = valid_dataset[544]
# image, label, ground_bbox = valid_dataset[292]
item = image.unsqueeze(0).to(device)
orig_img = unnormalize(image)

In [ ]:
get_cam(models_list, image, ground_bbox, item, start_epochs, step_epochs)

In [ ]:
get_grad_cam(models_list, image, ground_bbox, item, start_epochs, step_epochs)

In [27]:
def get_center_distance(bbox1, bbox2):
    """두 Bounding Box의 중심점 간의 유클리드 거리를 계산합니다."""
    if bbox1 is None or bbox2 is None:
        return float('inf')
    
    # 각 bbox의 중심점 계산
    cx1 = (bbox1[0] + bbox1[2]) / 2.0
    cy1 = (bbox1[1] + bbox1[3]) / 2.0
    cx2 = (bbox2[0] + bbox2[2]) / 2.0
    cy2 = (bbox2[1] + bbox2[3]) / 2.0
    
    # 유클리드 거리 반환
    return math.sqrt((cx1 - cx2)**2 + (cy1 - cy2)**2)

def evaluate_pointing_game(grad_cam_map, gt_bbox):
    """Grad-CAM 히트맵의 최대 활성화 지점이 정답 bbox 안에 있는지 평가합니다."""
    if gt_bbox is None:
        return False, (0, 0)
    
    # 2D 히트맵에서 최댓값을 가지는 좌표(y, x) 추출
    y_peak, x_peak = np.unravel_index(np.argmax(grad_cam_map), grad_cam_map.shape)
    
    x_min, y_min, x_max, y_max = gt_bbox
    # 최대 활성화 지점이 정답 상자 내부에 있는지 확인 (Hit 판별)
    is_hit = (x_min <= x_peak <= x_max) and (y_min <= y_peak <= y_max)
    
    return is_hit, (x_peak, y_peak)

In [31]:
def save_grid_results(model, viz_image, ground_bbox, model_input, idx, modes=['overlay', 'iou', 'center_dist', 'pointing', 'combined']):
    base_dir = "img"
    # 전달받은 modes에 대해서만 폴더 생성
    for mode in modes:
        os.makedirs(os.path.join(base_dir, f"grad_cam_{mode}"), exist_ok=True)

    target_layers = [name for name, module in model.named_modules() if isinstance(module, torch.nn.Conv2d)]
    orig_img = unnormalize(viz_image)
    ground_truth = [int(x) for x in ground_bbox]
    
    for mode in modes:
        if mode == 'combined':
            # 행(레이어 수) x 열(4가지 지표)
            fig, axes = plt.subplots(len(target_layers), 4, figsize=(16, 3 * len(target_layers)))
            
            # 맨 위 열 제목 설정
            col_titles = ['1. Overlay', '2. IoU', '3. Center Distance', '4. Pointing Game']
            for c in range(4):
                axes[0, c].set_title(col_titles[c], fontsize=14, fontweight='bold', pad=10)

            for i, layer_name in enumerate(target_layers):
                try:
                    grad_cam = generate_grad_cam(model, model_input, activation_layer=layer_name)
                    grad_cam_resized = cv2.resize(grad_cam, (orig_img.shape[1], orig_img.shape[0]))
                    grad_cam_bbox = get_bbox(grad_cam_resized, threshold=0.5)
                    
                    # 1. Overlay (첫 번째 열)
                    axes[i, 0].imshow(orig_img)
                    axes[i, 0].imshow(grad_cam_resized, cmap='jet', alpha=0.5)
                    axes[i, 0].set_ylabel(f"{layer_name}", fontsize=12, fontweight='bold') # 왼쪽에 레이어 이름 표시
                    axes[i, 0].set_xticks([])
                    axes[i, 0].set_yticks([])
                    
                    # 2. IoU (두 번째 열)
                    iou_score = get_iou(grad_cam_bbox, ground_truth)
                    res_img_iou = visualize_both_bbox_on_image(orig_img.copy(), grad_cam_bbox, ground_truth)
                    axes[i, 1].imshow(res_img_iou)
                    axes[i, 1].set_title(f"IOU: {iou_score:.3f}", fontsize=11)
                    axes[i, 1].axis("off")
                    
                    # 3. Center Distance (세 번째 열)
                    dist = get_center_distance(grad_cam_bbox, ground_truth)
                    res_img_dist = visualize_both_bbox_on_image(orig_img.copy(), grad_cam_bbox, ground_truth)
                    axes[i, 2].imshow(res_img_dist)
                    if grad_cam_bbox is not None:
                        axes[i, 2].plot((grad_cam_bbox[0]+grad_cam_bbox[2])/2, (grad_cam_bbox[1]+grad_cam_bbox[3])/2, 'ro', markersize=6)
                    axes[i, 2].plot((ground_truth[0]+ground_truth[2])/2, (ground_truth[1]+ground_truth[3])/2, 'go', markersize=6)
                    axes[i, 2].set_title(f"Dist: {dist:.1f}px", fontsize=11)
                    axes[i, 2].axis("off")

                    # 4. Pointing Game (네 번째 열)
                    is_hit, (peak_x, peak_y) = evaluate_pointing_game(grad_cam_resized, ground_truth)
                    hit_text, color = ("HIT", 'green') if is_hit else ("MISS", 'red')
                    axes[i, 3].imshow(orig_img)
                    rect = plt.Rectangle((ground_truth[0], ground_truth[1]), ground_truth[2]-ground_truth[0], ground_truth[3]-ground_truth[1], fill=False, edgecolor='green', linewidth=2)
                    axes[i, 3].add_patch(rect)
                    axes[i, 3].plot(peak_x, peak_y, marker='*', color=color, markersize=12)
                    axes[i, 3].set_title(hit_text, fontsize=11, color=color, fontweight='bold')
                    axes[i, 3].axis("off")

                except Exception:
                    for c in range(4):
                        axes[i, c].text(0.5, 0.5, "Error", ha='center')
                        axes[i, c].axis("off")

            plt.tight_layout()
            fig.subplots_adjust(top=0.97)
            save_path = os.path.join(base_dir, f"grad_cam_{mode}", f"{idx}.png")
            plt.savefig(save_path, bbox_inches='tight')
            plt.close(fig)

        else:
            num_plots = len(target_layers) + 1
            cols = 7
            rows = math.ceil(num_plots / cols)
            fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
            axes_flat = axes.flatten()

            axes_flat[0].imshow(orig_img)
            axes_flat[0].set_title("Original / GT", fontsize=10)
            axes_flat[0].axis("off")

            for i, layer_name in enumerate(target_layers):
                ax = axes_flat[i + 1]
                try:
                    grad_cam = generate_grad_cam(model, model_input, activation_layer=layer_name)
                    grad_cam_resized = cv2.resize(grad_cam, (orig_img.shape[1], orig_img.shape[0]))
                    
                    if mode == 'overlay':
                        ax.imshow(orig_img)
                        ax.imshow(grad_cam_resized, cmap='jet', alpha=0.5)
                        ax.set_title(layer_name, fontsize=8)
                    elif mode == 'iou':
                        grad_cam_bbox = get_bbox(grad_cam_resized, threshold=0.5)
                        iou_score = get_iou(grad_cam_bbox, ground_truth)
                        res_img = visualize_both_bbox_on_image(orig_img.copy(), grad_cam_bbox, ground_truth)
                        ax.imshow(res_img)
                        ax.set_title(f"{layer_name}\nIOU: {iou_score:.3f}", fontsize=8)
                    elif mode == 'center_dist':
                        grad_cam_bbox = get_bbox(grad_cam_resized, threshold=0.5)
                        dist = get_center_distance(grad_cam_bbox, ground_truth)
                        res_img = visualize_both_bbox_on_image(orig_img.copy(), grad_cam_bbox, ground_truth)
                        ax.imshow(res_img)
                        if grad_cam_bbox is not None:
                            ax.plot((grad_cam_bbox[0]+grad_cam_bbox[2])/2, (grad_cam_bbox[1]+grad_cam_bbox[3])/2, 'ro', markersize=4)
                        ax.plot((ground_truth[0]+ground_truth[2])/2, (ground_truth[1]+ground_truth[3])/2, 'go', markersize=4)
                        ax.set_title(f"{layer_name}\nDist: {dist:.1f}px", fontsize=8)
                    elif mode == 'pointing':
                        is_hit, (peak_x, peak_y) = evaluate_pointing_game(grad_cam_resized, ground_truth)
                        hit_text = "HIT" if is_hit else "MISS"
                        color = 'green' if is_hit else 'red'
                        ax.imshow(orig_img)
                        rect = plt.Rectangle((ground_truth[0], ground_truth[1]), ground_truth[2]-ground_truth[0], ground_truth[3]-ground_truth[1], fill=False, edgecolor='green', linewidth=2)
                        ax.add_patch(rect)
                        ax.plot(peak_x, peak_y, marker='*', color=color, markersize=10)
                        ax.set_title(f"{layer_name}\n{hit_text}", fontsize=8)
                        
                except Exception:
                    ax.text(0.5, 0.5, "Error", ha='center')
                ax.axis("off")

            for j in range(num_plots, len(axes_flat)):
                axes_flat[j].axis("off")

            plt.tight_layout()
            fig.subplots_adjust(top=0.97)
            
            save_path = os.path.join(base_dir, f"grad_cam_{mode}", f"{idx}.png")
            plt.savefig(save_path, bbox_inches='tight')
            plt.close(fig)

In [29]:
model_path = './models/cam_model_pre_e3.pt'
    
model = torch.load(model_path,  map_location=device, weights_only=False)

In [ ]:
# --- 1. 설정 모드 선택 ---
execution_mode = 'range' 
# 수행할 분석 지표를 리스트로 전달합니다. 필요한 것만 남기면 됩니다.
selected_modes = ['overlay', 'iou', 'center_dist', 'pointing', 'combined']

# 만약 4가지가 합쳐진 'combined' 파일 1개만 딱 뽑고 싶다면 이렇게 설정합니다.
# selected_modes = ['combined']

# --- 2. 모드별 세부 설정 ---
start_idx, end_idx = 540, 600  
num_random_samples = 50        

all_indices = list(range(len(valid_dataset)))

if execution_mode == 'all':
    indices_to_process = all_indices
    print(f"전체 데이터({len(indices_to_process)}개)를 처리합니다.")

elif execution_mode == 'range':
    indices_to_process = all_indices[max(0, start_idx) : min(len(all_indices), end_idx)]
    print(f"{start_idx}번부터 {end_idx-1}번까지 {len(indices_to_process)}개의 데이터를 처리합니다.")

elif execution_mode == 'random':
    indices_to_process = random.sample(all_indices, min(num_random_samples, len(all_indices)))
    print(f"무작위로 {len(indices_to_process)}개의 데이터를 추출하여 처리합니다: {indices_to_process}")

# --- 3. 모델 준비 및 루프 실행 ---
device = next(model.parameters()).device
model.eval()

for idx in tqdm(indices_to_process, desc=f"Mode: {execution_mode}"):
    image, label, ground_bbox = valid_dataset[idx]
    item = image.unsqueeze(0).to(device)
    save_grid_results(model, image, ground_bbox, item, idx, modes=selected_modes)
    
print("작업이 완료되었습니다.")

### 이미지를 출력한 상태로 파일을 업로드시 노트북 용량이 너무 크기 때문에 변경
### 1.  단일객체
[CAM 단일 객체](./img/cam_1.png)  
[Grad CAM 단일 객체](./img/grad_cam_1.png)  

### 2. 여러 객체
[CAM 여러 객체](./img/cam_2.png)  
[Grad 여러 단일 객체](./img/grad_cam_2.png)

### 3. 먼 거리 객체
[CAM 먼 거리 객체](./img/cam_3.png)  
[Grad CAM 먼 거리 객체](./img/grad_cam_3.png)

### IoU가 지역성과 모델의 성능을 판단하기 위한 절대적인 지표가 될 수 있는가?
1. IoU는 모델이 "물체가 어디에 있는지"를 얼마나 잘 찾아냈는지(지역성)를 평가하기에는 훌륭한 지표입니다. 하지만 IoU가 높다고 해서 반드시 '성능이 좋은 모델'이라고 단정할 수는 없습니다.

    - 이유: 모델의 최종 목적이 '분류'라면, 객체의 전체 영역을 대충 훑는 것보다 핵심적인 특징(Discriminative parts)을 정확히 짚어내는 것이 분류 정확도를 높이는 데 유리하기 때문입니다.


2. Grad-CAM 레이어 비교 분석
이미지 상의 Ep 4 결과를 바탕으로 비교해 보겠습니다.

- 레이어 3 (IoU: 0.6809):   
    특징: 히트맵이 치와와의 몸 전체와 얼굴을 비교적 넓게 커버하고 있습니다.  
    평가: 객체의 기하학적 형태를 잘 반영하고 있어 지역성 지표(IoU)는 높게 나타납니다. 하지만 분류를 위한 핵심 특징 외에 배경이나 덜 중요한 부위까지 포함하고 있을 가능성이 큽니다.

- 레이어 4 (IoU: 0.1623):  
    특징: 히트맵이 치와와의 눈과 코 등 '얼굴' 부위에만 매우 집중(Focus)되어 있습니다.  
    평가: 객체 전체를 덮지 못해 IoU는 낮게 측정됩니다. 그러나 모델이 이 동물을 '치와와'로 판단하게 만든 결정적인 근거는 바로 이 얼굴 부위입니다. 즉, 분류를 위한 추상화 수준은 더 높다고 볼 수 있습니다. 

[참고 자료](./img/grad_cam_alternative_544.png) 

### IoU가 절대적인 지표가 될 수 없다면 어떤 지표를 사용하면 되는가? 

- DIoU (Distance): 두 박스의 중심점 거리를 직접 계산에 포함해 수렴 속도를 높임.

- Point Detection: 중심점이나 특정 핵심 포인트가 정답 영역 내에 위치하는지를 판별하여 박스 전체의 겹침 정도와 무관하게 타겟 포착 여부를 직관적으로 측정함

- GIoU (Generalized): 겹치지 않아도 두 박스를 포함하는 최소 크기의 박스를 이용해 거리를 계산함.

- CIoU (Complete): 중첩 영역, 중심점 거리, 그리고 종횡비까지 모두 고려한 끝판왕 지표.